## Precomputation for $\kappa_{GMF}$

Precompute the values of $\kappa_{GMF}$, which defines the deflection between the UHECR arrived at earth $\omega$ and those after back propagation in the galactic magnetic field (GMF) $\omega_{GMF}$. This is precomputed for each composition and stored in the relevant group in `UHECRdata.h5`.

**NOTE:** This only needs to be computed once. Eventually we will set up a flag so that this would not compute when it is already computed.

In [1]:
import os
import h5py
import numpy as np

from fancy import Uhecr
from fancy.interfaces.stan import coord_to_uv

uhecr_file = "UHECRdata.h5"

# type of detector / UHECR dataset
# choose from ["TA2015", "auger2014", "auger2010"]
uhecr_label = "TA2015"

# initialize the Uhecr object
uhecr = Uhecr()

In [2]:
with h5py.File(uhecr_file, "r+") as f:
    print(f.keys())
    del f[uhecr_label]["kappa_gmf"]

<KeysViewHDF5 ['TA2015', 'auger2010', 'auger2014']>


In [3]:
'''Open the UHECR file'''
with h5py.File(uhecr_file, 'r+') as f:

    data = f[uhecr_label]
    
    uhecr.label = uhecr_label
    uhecr.year = data['year'][()]
    uhecr.day = data['day'][()]
    uhecr.zenith_angle = np.deg2rad(data['theta'][()])
    uhecr.energy = data['energy'][()]
    uhecr.N = len(uhecr.energy)
    glon = data['glon'][()]
    glat = data['glat'][()]
    uhecr.coord = uhecr.get_coordinates(glon, glat)

    uhecr.unit_vector = coord_to_uv(uhecr.coord)
    uhecr.period = uhecr._find_period()
    uhecr.A = uhecr._find_area(1.)

In [4]:
'''Build the kappa_gmf tables for all particles given in uhecr.nuc_tables'''
uhecr.build_kappa_gmf(uhecr_file, particle_type="all")

Precomputing kappa_gmf for each composition: 100%|██████████| 9/9 [00:42<00:00,  4.68s/it]


In [5]:
'''Check if we computed it correctly'''
with h5py.File(uhecr_file, "r") as f:
    print(f[uhecr_label].keys())

    print("kappa_gmf keys:")
    print(f[uhecr_label]["kappa_gmf"].keys())
    print(f[uhecr_label]["kappa_gmf"]["p"].keys())

    print(f[uhecr_label]["kappa_gmf"]["p"]["kappa_gmf"][()].shape)

<KeysViewHDF5 ['day', 'dec', 'energy', 'glat', 'glon', 'kappa_gmf', 'ra', 'theta', 'year']>
kappa_gmf keys:
<KeysViewHDF5 ['C', 'Fe', 'H', 'He', 'Li', 'N', 'O', 'Si', 'p']>
<KeysViewHDF5 ['kappa_gmf', 'omega_gal', 'omega_rand', 'omega_true']>
(72,)
